# Batch & Streaming Load - Demo

**Cel szkoleniowy:** Opanowanie metod ładowania danych: COPY INTO, Auto Loader i Structured Streaming.

**Zakres tematyczny:**
- COPY INTO: kiedy używać, parametry (FILEFORMAT, VALIDATION_MODE, PATTERN)
- Auto Loader (CloudFiles): file notification, checkpointing, schema inference
- Schema evolution w praktyce
- Structured Streaming: micro-batch architecture
- readStream() / writeStream()
- Triggering: once vs processingTime
- Zarządzanie checkpointami
- MERGE na streamingu

## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 2 - Lakehouse & Delta Lake
- **Typ notebooka**: Demo
- **Wymagania techniczne**:
  - Databricks Runtime 13.0+ (zalecane: 14.3 LTS)
  - Unity Catalog włączony
  - Uprawnienia: CREATE TABLE, CREATE SCHEMA, SELECT, MODIFY
  - Klaster: Standard z minimum 2 workers

## Wstęp teoretyczny

**Cel sekcji:** Zrozumienie różnych metod ładowania danych do Delta Lake: batch vs streaming.

**Podstawowe pojęcia:**
- **COPY INTO**: SQL command dla batch loads z idempotency (incremental batch)
- **Auto Loader**: Databricks-managed solution dla incremental file ingestion z automatycznym schema inference
- **Structured Streaming**: Spark streaming API z micro-batch processing i exactly-once semantics
- **Checkpoint**: Location przechowujący offset/progress dla fault tolerance

**Dlaczego to ważne?**
Wybór metody ingestion ma wpływ na latency, throughput, cost i operacyjną złożoność. COPY INTO dla batch (hourly/daily), Auto Loader dla near real-time z małymi plikami, Structured Streaming dla pure streaming sources (Kafka, Event Hub).

## Izolacja per użytkownik

Uruchom skrypt inicjalizacyjny dla per-user izolacji katalogów i schematów:

In [ ]:
%run ../00_setup

## Konfiguracja

Import bibliotek i ustawienie zmiennych środowiskowych:

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import datetime, timedelta
import time

# Wyświetl kontekst użytkownika
print("=== Kontekst użytkownika ===")
print(f"Katalog: {CATALOG}")
print(f"Schema Bronze: {BRONZE_SCHEMA}")
print(f"Użytkownik: {raw_user}")

# Ustaw katalog i schemat
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

# Ścieżki do danych i checkpointów
ORDERS_JSON = f"{DATASET_BASE_PATH}/orders/orders_batch.json"
CHECKPOINT_PATH = f"/tmp/{raw_user}/checkpoints"

print(f"\n=== Konfiguracja ===")
print(f"Orders JSON: {ORDERS_JSON}")
print(f"Checkpoint path: {CHECKPOINT_PATH}")

---

## Sekcja 1: COPY INTO - Batch Ingestion

**Wprowadzenie teoretyczne:**

COPY INTO to SQL command dla idempotent batch loads. Automatycznie śledzi załadowane pliki i pomija duplikaty. Idealny dla scheduled batch jobs (hourly, daily).

**Kluczowe pojęcia:**
- **Idempotency**: Wielokrotne wykonanie COPY INTO z tymi samymi plikami nie powoduje duplikatów
- **File tracking**: Delta Log przechowuje listę załadowanych plików
- **Pattern matching**: Możliwość filtrowania plików po nazwie (PATTERN)
- **Validation mode**: Kontrola zachowania przy błędach (PERMISSIVE, FAILFAST)

**Zastosowanie praktyczne:**
- Scheduled batch loads z cloud storage (S3, ADLS, GCS)
- Incremental data ingestion bez ręcznego trackowania offsetów
- ETL pipelines z retry logic

### Przykład 1.1: Podstawowy COPY INTO

**Cel:** Załadowanie plików JSON za pomocą COPY INTO

In [ ]:
# Przykład 1.1 - COPY INTO basic

# Utwórz target table
copy_into_table = f"{BRONZE_SCHEMA}.orders_copy_into"

spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {copy_into_table} (
        order_id INT,
        customer_id INT,
        order_date STRING,
        order_amount DOUBLE,
        order_status STRING,
        _metadata STRUCT<file_path: STRING, file_name: STRING, file_modification_time: TIMESTAMP>
    )
    USING DELTA
""")

print(f"✓ Utworzono target table: {copy_into_table}")

# Wykonaj COPY INTO
copy_result = spark.sql(f"""
    COPY INTO {copy_into_table}
    FROM (SELECT *, _metadata FROM '{ORDERS_JSON}')
    FILEFORMAT = JSON
    FORMAT_OPTIONS ('multiLine' = 'true')
    COPY_OPTIONS ('mergeSchema' = 'true')
""")

print("\n=== COPY INTO Result ===")
display(copy_result)

# Sprawdź załadowane dane
loaded_count = spark.table(copy_into_table).count()
print(f"\nZaładowano {loaded_count} rekordów")

# Wyświetl przykładowe dane z metadata
print("\n=== Dane z metadata ===")
display(
    spark.table(copy_into_table)
    .select("order_id", "order_amount", "_metadata.file_name")
    .limit(5)
)

**Wyjaśnienie:**

COPY INTO:
- **_metadata column**: Automatycznie dodawana kolumna z file path, name, modification time
- **Idempotency**: Ponowne wykonanie tego samego COPY INTO nie załaduje duplikatów
- **mergeSchema**: Automatyczne dodawanie nowych kolumn (schema evolution)
- **File tracking**: Delta Log przechowuje hash załadowanych plików

### Przykład 1.2: COPY INTO z filtrowaniem (PATTERN)

**Cel:** Selective ingestion - tylko pliki spełniające pattern

In [ ]:
# Przykład 1.2 - COPY INTO z PATTERN

# Re-run tego samego COPY INTO - demonstracja idempotency
print("=== Ponowne wykonanie COPY INTO (idempotency test) ===")
copy_result_2 = spark.sql(f"""
    COPY INTO {copy_into_table}
    FROM (SELECT *, _metadata FROM '{ORDERS_JSON}')
    FILEFORMAT = JSON
    FORMAT_OPTIONS ('multiLine' = 'true')
""")

display(copy_result_2)

# Sprawdź czy count się nie zmienił (idempotency)
new_count = spark.table(copy_into_table).count()
print(f"\nLiczba rekordów (po ponownym COPY INTO): {new_count}")
print(f"Czy idempotentny? {new_count == loaded_count}")

# Historia COPY INTO
print("\n=== Historia COPY INTO ===")
history = spark.sql(f"DESCRIBE HISTORY {copy_into_table}")
display(
    history
    .filter(F.col("operation") == "COPY INTO")
    .select("version", "timestamp", "operation", "operationMetrics")
)

---

## Sekcja 2: Auto Loader (CloudFiles)

**Wprowadzenie teoretyczne:**

Auto Loader to Databricks-managed solution dla incremental file ingestion. Używa file notification (Event Grid/SQS) lub directory listing dla automatic discovery nowych plików. Idealny dla near real-time ingestion z małymi plikami.

**Kluczowe pojęcia:**
- **cloudFiles format**: Specjalny format Spark dla Auto Loader
- **Schema inference**: Automatyczne wykrywanie i ewolucja schematu
- **Checkpoint location**: Przechowuje progress i schema history
- **File notification**: Event-driven approach dla cloud storage

**Zastosowanie praktyczne:**
- Near real-time ingestion (latency: sekundy-minuty)
- Małe pliki arriving continuously
- Schema evolution bez manual intervention

### Przykład 2.1: Auto Loader - Basic Setup

**Cel:** Konfiguracja Auto Loader z schema inference

In [ ]:
# Przykład 2.1 - Auto Loader basic

autoloader_table = f"{BRONZE_SCHEMA}.orders_autoloader"
autoloader_checkpoint = f"{CHECKPOINT_PATH}/autoloader_orders"

# Auto Loader z readStream
orders_stream = (
    spark.readStream
    .format("cloudFiles")  # Auto Loader format
    .option("cloudFiles.format", "json")  # Source format
    .option("cloudFiles.schemaLocation", f"{autoloader_checkpoint}/schema")  # Schema tracking
    .option("cloudFiles.inferColumnTypes", "true")  # Infer types (not just strings)
    .option("multiLine", "true")
    .load(ORDERS_JSON)
)

print("=== Auto Loader Stream Schema ===")
orders_stream.printSchema()

# Zapis z trigger(once) dla demo (batch mode)
query = (
    orders_stream
    .writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", autoloader_checkpoint)
    .trigger(once=True)  # Process all available data, then stop
    .table(autoloader_table)
)

# Czekaj na zakończenie
query.awaitTermination()

print(f"\n✓ Auto Loader completed")
print(f"Załadowano {spark.table(autoloader_table).count()} rekordów")

# Wyświetl dane
print("\n=== Załadowane dane ===")
display(spark.table(autoloader_table).limit(5))

**Wyjaśnienie:**

Auto Loader:
- **cloudFiles format**: Specjalny format dla Structured Streaming
- **trigger(once=True)**: Batch mode - process all files, then stop (użyteczne dla testing)
- **checkpointLocation**: Obowiązkowe - przechowuje progress i schema
- **Schema inference**: Automatyczne wykrywanie typów (inferColumnTypes=true)

W produkcji: używamy `trigger(processingTime='5 minutes')` dla continuous processing.

### Przykład 2.2: Auto Loader - Schema Evolution

**Cel:** Demonstracja automatycznej ewolucji schematu przy nowych plikach

In [ ]:
# Przykład 2.2 - Schema Evolution

# Sprawdź schema location (gdzie Auto Loader przechowuje schema history)
schema_location = f"{autoloader_checkpoint}/schema"
print(f"Schema location: {schema_location}")

# Lista plików w schema location
print("\n=== Schema history files ===")
schema_files = dbutils.fs.ls(schema_location)
for file in schema_files:
    print(f"  {file.name}")

# Odczytaj schema history
print("\n=== Current Schema ===")
current_schema = spark.table(autoloader_table).schema
for field in current_schema.fields:
    print(f"  {field.name}: {field.dataType}")

# W przypadku nowych plików z dodatkowymi kolumnami,
# Auto Loader automatycznie zaktualizuje schemat
print("\n⚠️ Uwaga: Schema evolution działa automatycznie przy nowych plikach z dodatkowymi kolumnami")

---

## Sekcja 3: Structured Streaming - Continuous Processing

**Wprowadzenie teoretyczne:**

Structured Streaming to Spark API dla continuous data processing. Traktuje stream jako unbounded table z micro-batch execution. Zapewnia exactly-once semantics i fault tolerance.

**Kluczowe pojęcia:**
- **readStream / writeStream**: API dla streaming operations
- **Trigger**: Processing interval (once, processingTime, availableNow)
- **Output mode**: append, complete, update
- **Watermark**: Time-based windowing dla late data handling

**Zastosowanie praktyczne:**
- Real-time ETL z Kafka, Event Hub, Kinesis
- Continuous aggregations i windowing
- Exactly-once processing semantics

### Przykład 3.1: Structured Streaming - Basic Stream

**Cel:** Utworzenie basic streaming pipeline z transformacjami

In [ ]:
# Przykład 3.1 - Structured Streaming basic

streaming_table = f"{BRONZE_SCHEMA}.orders_streaming"
streaming_checkpoint = f"{CHECKPOINT_PATH}/streaming_orders"

# ReadStream z transformacjami
orders_stream = (
    spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "json")
    .option("cloudFiles.schemaLocation", f"{streaming_checkpoint}/schema")
    .option("multiLine", "true")
    .load(ORDERS_JSON)
)

# Transformacje na streamie (jak na batch DataFrame)
orders_transformed = (
    orders_stream
    .withColumn("order_date", F.to_date(F.col("order_date")))
    .withColumn("order_status", F.upper(F.col("order_status")))
    .withColumn("stream_processed_ts", F.current_timestamp())
    .filter(F.col("order_amount") > 0)  # Quality check
)

print("=== Transformed Stream Schema ===")
orders_transformed.printSchema()

# WriteStream z trigger(once) dla demo
query = (
    orders_transformed
    .writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", streaming_checkpoint)
    .trigger(once=True)
    .table(streaming_table)
)

# Czekaj na zakończenie
query.awaitTermination()

print(f"\n✓ Streaming pipeline completed")
print(f"Przetworzono {spark.table(streaming_table).count()} rekordów")

# Wyświetl dane z transformation
print("\n=== Przetworzone dane ===")
display(
    spark.table(streaming_table)
    .select("order_id", "order_date", "order_status", "stream_processed_ts")
    .limit(5)
)

**Wyjaśnienie:**

Structured Streaming:
- **Transformacje**: Możemy używać standardowych DataFrame API (withColumn, filter, join)
- **trigger(once=True)**: Batch mode - użyteczne dla testing i backfill
- **outputMode="append"**: Tylko nowe rekordy zapisywane (domyślne dla streaming)
- **checkpointLocation**: Fault tolerance - możliwość recovery po failure

### Przykład 3.2: Streaming Aggregations

**Cel:** Continuous aggregations na streaming data

In [ ]:
# Przykład 3.2 - Streaming Aggregations

streaming_agg_table = f"{BRONZE_SCHEMA}.orders_streaming_agg"
agg_checkpoint = f"{CHECKPOINT_PATH}/streaming_agg"

# ReadStream
orders_stream = (
    spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "json")
    .option("cloudFiles.schemaLocation", f"{agg_checkpoint}/schema")
    .option("multiLine", "true")
    .load(ORDERS_JSON)
)

# Agregacje: count i sum per status
orders_agg = (
    orders_stream
    .withColumn("order_status", F.upper(F.col("order_status")))
    .groupBy("order_status")
    .agg(
        F.count("order_id").alias("total_orders"),
        F.sum("order_amount").alias("total_revenue"),
        F.avg("order_amount").alias("avg_order_value")
    )
)

# WriteStream z outputMode="complete" dla agregacji
query = (
    orders_agg
    .writeStream
    .format("delta")
    .outputMode("complete")  # Complete mode dla groupBy bez watermark
    .option("checkpointLocation", agg_checkpoint)
    .trigger(once=True)
    .table(streaming_agg_table)
)

query.awaitTermination()

print(f"\n✓ Streaming aggregation completed")
print("\n=== Wyniki agregacji ===")
display(spark.table(streaming_agg_table))

**Wyjaśnienie:**

Streaming Aggregations:
- **outputMode="complete"**: Cała tabela wynikowa zapisywana przy każdym micro-batch (wymagane dla groupBy bez watermark)
- **outputMode="update"**: Tylko zaktualizowane wiersze (użyteczne z watermark)
- **Stateful operations**: GroupBy/Aggregations wymagają state management (przechowywany w checkpoint)

W produkcji: używamy watermark dla windowed aggregations i outputMode="update".

---

## Sekcja 4: MERGE na Streamingu (Upsert)

**Wprowadzenie teoretyczne:**

Structured Streaming może pisać do Delta z MERGE logic (upsert). Używamy foreachBatch dla custom write logic w każdym micro-batch.

**Zastosowanie:**
- CDC (Change Data Capture) streaming
- Upsert streaming events do dimension tables
- Deduplication w real-time

### Przykład 4.1: Streaming MERGE (Upsert)

**Cel:** Implementacja streaming upsert z MERGE INTO

In [ ]:
# Przykład 4.1 - Streaming MERGE

from delta.tables import DeltaTable

# Target table dla upsert
upsert_table = f"{BRONZE_SCHEMA}.orders_upsert"
upsert_checkpoint = f"{CHECKPOINT_PATH}/streaming_upsert"

# Utwórz target table jeśli nie istnieje
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {upsert_table} (
        order_id INT,
        customer_id INT,
        order_date STRING,
        order_amount DOUBLE,
        order_status STRING,
        last_updated TIMESTAMP
    )
    USING DELTA
""")

# ForeachBatch function dla MERGE
def upsert_to_delta(microBatchDF, batchId):
    # Dodaj last_updated timestamp
    microBatchDF = microBatchDF.withColumn("last_updated", F.current_timestamp())
    
    # DeltaTable dla MERGE
    deltaTable = DeltaTable.forName(spark, upsert_table)
    
    # MERGE INTO logic
    (
        deltaTable.alias("target")
        .merge(
            microBatchDF.alias("source"),
            "target.order_id = source.order_id"
        )
        .whenMatchedUpdate(set = {
            "order_status": "source.order_status",
            "order_amount": "source.order_amount",
            "last_updated": "source.last_updated"
        })
        .whenNotMatchedInsert(values = {
            "order_id": "source.order_id",
            "customer_id": "source.customer_id",
            "order_date": "source.order_date",
            "order_amount": "source.order_amount",
            "order_status": "source.order_status",
            "last_updated": "source.last_updated"
        })
        .execute()
    )
    
    print(f"Batch {batchId}: Merged {microBatchDF.count()} records")

# ReadStream
orders_stream = (
    spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "json")
    .option("cloudFiles.schemaLocation", f"{upsert_checkpoint}/schema")
    .option("multiLine", "true")
    .load(ORDERS_JSON)
)

# WriteStream z foreachBatch
query = (
    orders_stream
    .writeStream
    .foreachBatch(upsert_to_delta)  # Custom MERGE logic
    .option("checkpointLocation", upsert_checkpoint)
    .trigger(once=True)
    .start()
)

query.awaitTermination()

print(f"\n✓ Streaming MERGE completed")
print(f"Final count: {spark.table(upsert_table).count()}")

print("\n=== Upserted data ===")
display(spark.table(upsert_table).orderBy("order_id").limit(10))

**Wyjaśnienie:**

Streaming MERGE:
- **foreachBatch**: Custom function wykonywana na każdym micro-batch
- **MERGE INTO**: Upsert logic - UPDATE jeśli istnieje, INSERT jeśli nie
- **Idempotency**: Ponowne procesowanie tego samego batch ID daje ten sam rezultat
- **Use case**: CDC streaming, real-time dimension updates, deduplication

---

## Porównanie metod ingestion

| Feature | COPY INTO | Auto Loader | Structured Streaming |
|---------|-----------|-------------|---------------------|
| **Latency** | Minuty-godziny | Sekundy-minuty | Sub-sekundy |
| **Use case** | Scheduled batch | Near real-time files | Pure streaming (Kafka) |
| **Idempotency** | ✅ Built-in | ✅ Built-in | ⚠️ Requires checkpoint |
| **Schema evolution** | ⚠️ Manual | ✅ Automatic | ⚠️ Manual |
| **Complexity** | Low | Medium | High |
| **Cost** | Lowest | Medium | Highest |
| **File tracking** | Delta Log | Checkpoint | Checkpoint |

**Rekomendacje:**
- **COPY INTO**: Batch loads (hourly, daily), duże pliki, niskie koszty
- **Auto Loader**: Near real-time, małe pliki, schema evolution
- **Structured Streaming**: Pure streaming sources (Kafka), sub-second latency

---

## Best Practices

**COPY INTO:**
- Używaj dla scheduled batch jobs (daily, hourly)
- Zawsze dodawaj _metadata column dla audytu
- Używaj PATTERN dla filtrowania plików
- Monitor operationMetrics w DESCRIBE HISTORY

**Auto Loader:**
- Włącz schema inference (inferColumnTypes=true)
- Używaj trigger(availableNow=True) dla backfill
- Monitor schema evolution w schemaLocation
- Rozważ file notification dla dużej liczby plików (>10k)

**Structured Streaming:**
- Zawsze ustawiaj checkpointLocation
- Używaj trigger(processingTime) dla continuous streams
- Implementuj watermark dla windowed aggregations
- Monitor stream metrics (numInputRows, inputRowsPerSecond)

**Checkpoints:**
- Przechowuj w niezależnej lokalizacji (nie w table location)
- Backup przed schema changes
- Nie usuwaj checkpoint - loss of progress!
- Używaj external location (S3, ADLS) dla durability

---

## Troubleshooting

**Problem 1: "Stream stopped unexpectedly"**
**Rozwiązanie:**
- Sprawdź checkpoint location - czy istnieje i jest writable
- Sprawdź logi streaming query: `query.lastProgress`
- Monitor exceptions: `query.exception()`

**Problem 2: "Schema mismatch in Auto Loader"**
**Rozwiązanie:**
```python
# Włącz schema evolution
.option("cloudFiles.schemaEvolutionMode", "addNewColumns")
.option("mergeSchema", "true")
```

**Problem 3: COPY INTO nie wykrywa nowych plików**
**Rozwiązanie:**
- COPY INTO śledzi tylko file path - zmiana zawartości nie jest wykrywana
- Użyj COPY_OPTIONS ('force' = 'true') dla re-ingestion

**Problem 4: Streaming aggregation state grows indefinitely**
**Rozwiązanie:**
```python
# Dodaj watermark dla time-based cleanup
.withWatermark("event_time", "1 hour")
```

---

## Podsumowanie

**W tym notebooku nauczyliśmy się:**

✅ **COPY INTO:**
- Idempotent batch loads z automatic file tracking
- Pattern matching dla selective ingestion
- _metadata column dla audytu

✅ **Auto Loader:**
- Near real-time file ingestion z cloudFiles format
- Automatic schema inference i evolution
- Checkpoint-based progress tracking

✅ **Structured Streaming:**
- Continuous processing z micro-batch architecture
- Transformacje i agregacje na streaming data
- foreachBatch dla custom write logic (MERGE)

✅ **Triggering modes:**
- trigger(once=True) - batch mode dla testing
- trigger(processingTime) - continuous processing
- trigger(availableNow=True) - backfill mode

**Kluczowe wnioski:**
1. Wybór metody ingestion zależy od latency requirements i source type
2. COPY INTO dla scheduled batch, Auto Loader dla near real-time files
3. Structured Streaming dla pure streaming sources (Kafka)
4. Checkpoint location jest krytyczny dla fault tolerance

**Następne kroki:**
- **Kolejny notebook**: 04_bronze_silver_gold_pipeline.ipynb
- **Warsztat praktyczny**: 02_ingestion_pipeline_workshop.ipynb
- **Delta Live Tables**: Declarative pipelines z automatic orchestration

---

## Cleanup

Posprzątaj zasoby utworzone podczas notebooka:

In [ ]:
# Opcjonalne czyszczenie zasobów testowych
# UWAGA: Uruchom tylko jeśli chcesz usunąć wszystkie utworzone dane

# spark.sql(f"DROP TABLE IF EXISTS {copy_into_table}")
# spark.sql(f"DROP TABLE IF EXISTS {autoloader_table}")
# spark.sql(f"DROP TABLE IF EXISTS {streaming_table}")
# spark.sql(f"DROP TABLE IF EXISTS {streaming_agg_table}")
# spark.sql(f"DROP TABLE IF EXISTS {upsert_table}")

# Usuń checkpointy
# dbutils.fs.rm(CHECKPOINT_PATH, recurse=True)

# spark.catalog.clearCache()
# print("Zasoby zostały wyczyszczone")